## Assignment 4

### Import Modules

In [176]:
import struct
import numpy as np
from array import array
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
from sklearn.metrics import confusion_matrix

### Functions

In [177]:
def readExcelSheet(excelfile):
    from pandas import read_excel
    return (read_excel(excelfile)).values

In [178]:
def readExcelRange(excelfile,sheetname="Sheet1",startrow=1,endrow=1,startcol=1,endcol=1):
    from pandas import read_excel
    values=(read_excel(excelfile, sheetname,header=None)).values;
    return values[startrow-1:endrow,startcol-1:endcol]

In [179]:
def readExcel(excelfile,**args):
    if args:
        data=readExcelRange(excelfile,**args)
    else:
        data=readExcelSheet(excelfile)
    if data.shape==(1,1):
        return data[0,0]
    elif (data.shape)[0]==1:
        return data[0]
    else:
        return data

In [180]:
def getSheetNames(excelfile):
    from pandas import ExcelFile
    return (ExcelFile(excelfile)).sheet_names

In [181]:
def writeExcelData(x,excelfile,sheetname,startrow,startcol):
    from pandas import DataFrame, ExcelWriter
    from openpyxl import load_workbook
    df=DataFrame(x)
    book = load_workbook(excelfile)
    writer = ExcelWriter(excelfile, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, sheet_name=sheetname,startrow=startrow-1, startcol=startcol-1, header=False, index=False)
    writer.save()
    writer.close()

### Prepare Data


In [182]:
excelfile='/Users/eve7947/Downloads/Hsing-Yi_Assignment_4_Data_and_Template.xlsx'

In [183]:
sheets=getSheetNames(excelfile);
sheets

['Training Data', 'Classifiers', 'To be classified', 'Performance', 'ID']

In [184]:
training_data=readExcel(excelfile)[:,0:15]

In [185]:
training_target=readExcel(excelfile)[:,15]

In [186]:
training_target_2=readExcel(excelfile)[:,16]

In [187]:
testing_data=readExcel(excelfile,sheetname="To be classified",startrow=5,endrow=54,startcol=1,endcol=15)

In [188]:
testing_data.shape

(50, 15)

In [189]:
type(training_data)

numpy.ndarray

In [190]:
X = np.asmatrix(training_data, dtype='float')

In [191]:
type(X)

numpy.matrixlib.defmatrix.matrix

In [192]:
X.shape

(6600, 15)

In [193]:
T = np.asmatrix(training_target, dtype='float')

In [194]:
T.shape

(1, 6600)

In [195]:
T = T.transpose()

In [196]:
T.shape

(6600, 1)

In [197]:
n,m = X.shape
X0 = np.ones((n,1))
Xa = np.hstack((X0,X))

In [198]:
Xa.shape

(6600, 16)

In [199]:
X_test = np.asmatrix(testing_data, dtype='float')

In [200]:
nt,mt = X_test.shape
X0_test = np.ones((nt,1))
Xa_test = np.hstack((X0_test,X_test))

In [201]:
Xa_test.shape

(50, 16)

In [202]:
T2 = np.array(training_target_2, dtype='float')

In [203]:
T2 = T2.transpose()

In [204]:
T2 = pd.get_dummies(T2)

In [205]:
T2 = np.asmatrix(T2)

In [206]:
T2.shape

(6600, 6)

### 2-Class Linear Classifier

In [226]:
Xainverse = np.linalg.pinv(Xa,rcond=1e-15)

In [227]:
w = np.dot(Xainverse, T)

In [228]:
w

matrix([[-2.94413221e-01],
        [ 7.32266635e-04],
        [ 1.08866787e-03],
        [ 5.54479443e-04],
        [ 6.23305624e-04],
        [ 2.58846750e-04],
        [ 8.87480321e-04],
        [ 1.15591260e-01],
        [ 5.40206261e-02],
        [ 1.24801335e-01],
        [-1.16321158e-01],
        [ 2.43438146e-01],
        [ 1.67296233e-01],
        [ 6.43946828e-02],
        [-3.06337864e-01],
        [ 5.36356402e-01]])

In [229]:
w.shape

(16, 1)

In [231]:
writeExcelData(w,excelfile,'Classifiers',5,1)

In [232]:
T_test = np.dot(Xa_test, w)

In [233]:
T_test = np.sign(T_test)

In [234]:
writeExcelData(T_test,excelfile,'To be classified',5,16)

### Multiple Classes Classifier

In [235]:
w2 = np.dot(Xainverse, T2)

In [236]:
w2.shape

(16, 6)

In [237]:
writeExcelData(w2,excelfile,'Classifiers',5,5)

In [238]:
T2_test = np.dot(Xa_test, w2)

In [239]:
T2_test = np.array(T2_test)

In [240]:
T2_test.shape

(50, 6)

In [241]:
T2_test = pd.DataFrame(T2_test)

In [242]:
T2_test = T2_test.idxmax(1)

In [243]:
T2_test = np.array(T2_test)

In [244]:
T2_test

array([1, 4, 0, 4, 4, 5, 0, 4, 5, 1, 2, 3, 1, 4, 0, 1, 5, 5, 1, 0, 5, 5,
       4, 5, 4, 3, 2, 4, 4, 4, 2, 0, 2, 2, 5, 1, 1, 5, 2, 5, 2, 4, 1, 5,
       1, 2, 5, 2, 4, 2])

In [245]:
writeExcelData(T2_test,excelfile,'To be classified',5,17)

### Test Performance

In [246]:
T3_test = np.dot(Xa, w)

In [247]:
T3_test.shape

(6600, 1)

In [248]:
T3_test = np.sign(T3_test)

In [249]:
m1 = confusion_matrix(T, T3_test)

In [250]:
writeExcelData(m1,'Performance',10,3)

TypeError: writeExcelData() missing 1 required positional argument: 'startcol'

In [251]:
T4_test = np.dot(Xa, w2)

In [252]:
T4_test.shape

(6600, 6)

In [253]:
T4_test = pd.DataFrame(T4_test)
T4_test = T4_test.idxmax(1)
T4_test = np.array(T4_test)

In [254]:
T4_test

array([4, 0, 2, ..., 2, 0, 5])

In [255]:
T4_test.shape

(6600,)

In [256]:
T4_test = np.asmatrix(T4_test)
T4_test = T4_test.transpose()

In [257]:
T4_test.shape

(6600, 1)

In [258]:
T2 = np.array(training_target_2, dtype='float')
T2 = T2.transpose()

In [259]:
m2 = confusion_matrix(T2, T4_test)

In [261]:
writeExcelData(m2,excelfile,'Performance',19,3)